# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

from pyspark import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt
from pyspark.sql.functions import udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit, when, col
from pyspark.sql.functions import col
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType

In [2]:
# Le pido que haga una operación cualquiera para comprobar que efectivamente he inicializado bien Spark.
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[135, 581, 198, 399, 905]

###### Preparación del dataset

In [2]:
#importar datos
liga=sc.textFile('partidosLigaNBA.csv')

In [3]:
liga.take(3)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93']

In [4]:
#eliminación de PTS y play-off. Separación de columnas por ':'

liga_split = liga.filter(lambda x: "PTS" not in x).filter(lambda x: "Playoffs" not in x).map(lambda x: x.split(":"))

In [5]:
df_intermedio = sqlContext.createDataFrame(liga_split)
df_intermedio.registerTempTable('interactions')

In [6]:
#Elimino columnas de hora
df2 = df_intermedio.drop('_2')
df2 = df2.drop('_3')



In [7]:
df2 = df2.select(col("_1").alias("fecha"), 
                 col("_4").alias("visitante"),
                 col("_5").alias("puntosVisitante"),
                 col("_6").alias("local"),
                 col("_7").alias("puntosLocal"))


In [8]:
#columna mes

df2 = df2.withColumn("mes y dia", expr("substring(fecha, 1, length(fecha)-8)"))

split_col = split(df2['mes y dia'], ',')
df2 = df2.withColumn('diasemana', split_col.getItem(0))
df2 = df2.withColumn('mes', split_col.getItem(1))

In [9]:
df2.show()

+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+-----+
|            fecha|           visitante|puntosVisitante|               local|puntosLocal|mes y dia|diasemana|  mes|
+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+-----+
|Tue, Oct 30, 2007|           Utah Jazz|            117|Golden State Warr...|         96|Tue, Oct |      Tue| Oct |
|Tue, Oct 30, 2007|     Houston Rockets|             95|  Los Angeles Lakers|         93|Tue, Oct |      Tue| Oct |
|Tue, Oct 30, 2007|Portland Trail Bl...|             97|   San Antonio Spurs|        106|Tue, Oct |      Tue| Oct |
|Wed, Oct 31, 2007|    Dallas Mavericks|             92| Cleveland Cavaliers|         74|Wed, Oct |      Wed| Oct |
|Wed, Oct 31, 2007| Seattle SuperSonics|            103|      Denver Nuggets|        120|Wed, Oct |      Wed| Oct |
|Wed, Oct 31, 2007|  Washington Wizards|            110|      Indiana Pa

In [13]:
#columna año
from pyspark.sql.functions import substring

df3 = df2.withColumn("año", substring(col("fecha"), -4, df2.count()))

In [14]:
#hay espacios en blanco en la columna 'mes'
from pyspark.sql.functions import trim
df3 = df3.withColumn("mes", trim(df3.mes))

In [15]:
df3.show()

+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+---+----+
|            fecha|           visitante|puntosVisitante|               local|puntosLocal|mes y dia|diasemana|mes| año|
+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+---+----+
|Tue, Oct 30, 2007|           Utah Jazz|            117|Golden State Warr...|         96|Tue, Oct |      Tue|Oct|2007|
|Tue, Oct 30, 2007|     Houston Rockets|             95|  Los Angeles Lakers|         93|Tue, Oct |      Tue|Oct|2007|
|Tue, Oct 30, 2007|Portland Trail Bl...|             97|   San Antonio Spurs|        106|Tue, Oct |      Tue|Oct|2007|
|Wed, Oct 31, 2007|    Dallas Mavericks|             92| Cleveland Cavaliers|         74|Wed, Oct |      Wed|Oct|2007|
|Wed, Oct 31, 2007| Seattle SuperSonics|            103|      Denver Nuggets|        120|Wed, Oct |      Wed|Oct|2007|
|Wed, Oct 31, 2007|  Washington Wizards|        

In [16]:
#cambio el tipo de dato de string a integerpara los puntos y así poder operar 
from pyspark.sql.types import IntegerType

df4 = df3.withColumn("puntosLocal", df2["puntosLocal"].cast(IntegerType()))


In [17]:
df4.dtypes

[('fecha', 'string'),
 ('visitante', 'string'),
 ('puntosVisitante', 'string'),
 ('local', 'string'),
 ('puntosLocal', 'int'),
 ('mes y dia', 'string'),
 ('diasemana', 'string'),
 ('mes', 'string'),
 ('año', 'string')]

In [18]:
df5 = df4.withColumn('puntosVisitante',df4['puntosVisitante'].cast(IntegerType()))

In [19]:
df5.dtypes

[('fecha', 'string'),
 ('visitante', 'string'),
 ('puntosVisitante', 'int'),
 ('local', 'string'),
 ('puntosLocal', 'int'),
 ('mes y dia', 'string'),
 ('diasemana', 'string'),
 ('mes', 'string'),
 ('año', 'string')]

In [20]:
df5.show()

+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+---+----+
|            fecha|           visitante|puntosVisitante|               local|puntosLocal|mes y dia|diasemana|mes| año|
+-----------------+--------------------+---------------+--------------------+-----------+---------+---------+---+----+
|Tue, Oct 30, 2007|           Utah Jazz|            117|Golden State Warr...|         96|Tue, Oct |      Tue|Oct|2007|
|Tue, Oct 30, 2007|     Houston Rockets|             95|  Los Angeles Lakers|         93|Tue, Oct |      Tue|Oct|2007|
|Tue, Oct 30, 2007|Portland Trail Bl...|             97|   San Antonio Spurs|        106|Tue, Oct |      Tue|Oct|2007|
|Wed, Oct 31, 2007|    Dallas Mavericks|             92| Cleveland Cavaliers|         74|Wed, Oct |      Wed|Oct|2007|
|Wed, Oct 31, 2007| Seattle SuperSonics|            103|      Denver Nuggets|        120|Wed, Oct |      Wed|Oct|2007|
|Wed, Oct 31, 2007|  Washington Wizards|        

In [21]:
#Creo una columna para determinar quién ganó el partido
df6 =df5.withColumn('ganador',when(
    (col("puntosVisitante") > col("puntosLocal")), col('visitante')).otherwise(col('local')))


In [30]:
df6.drop('mes y dia')
df6.drop('fecha').show()

+--------------------+---------------+--------------------+-----------+---------+---------+---+----+-------------------+
|           visitante|puntosVisitante|               local|puntosLocal|mes y dia|diasemana|mes| año|            ganador|
+--------------------+---------------+--------------------+-----------+---------+---------+---+----+-------------------+
|           Utah Jazz|            117|Golden State Warr...|         96|Tue, Oct |      Tue|Oct|2007|          Utah Jazz|
|     Houston Rockets|             95|  Los Angeles Lakers|         93|Tue, Oct |      Tue|Oct|2007|    Houston Rockets|
|Portland Trail Bl...|             97|   San Antonio Spurs|        106|Tue, Oct |      Tue|Oct|2007|  San Antonio Spurs|
|    Dallas Mavericks|             92| Cleveland Cavaliers|         74|Wed, Oct |      Wed|Oct|2007|   Dallas Mavericks|
| Seattle SuperSonics|            103|      Denver Nuggets|        120|Wed, Oct |      Wed|Oct|2007|     Denver Nuggets|
|  Washington Wizards|          

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

fuente: https://unraveldata.com/to-cache-or-not-to-cache/

El almacenamiento en caché de RDD en Spark es un mecanismo para acelerar las aplicaciones que acceden al mismo RDD varias veces. Un RDD que no se almacena en caché, ni se registra, se vuelve a evaluar cada vez que se invoca una acción en ese RDD. Hay dos llamadas a funciones para almacenar en caché un RDD: cache () y persist (nivel: StorageLevel). La diferencia entre ellos es que el caché () almacenará en caché el RDD en la memoria, mientras que persist puede almacenar en caché en la memoria, en el disco o en la memoria off-heap de acuerdo con la estrategia de almacenamiento en caché especificada por nivel.
Persist () sin argumento es equivalente a caché ().
Collect, por otra parte, hace referencia a la agrupación de la información distribuida entre varios worker, tras una operación de transformación que reduzca la información.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

1. Descarga datos de campus virtual.
2. Intento realizar la carga del archivo al entorno de Python, pero su gran tamaño (>25 MB) no permite realizar la acción.
3. Subo el archivo a la nube (mi Drive personal)
4. Importo datos a través de urllib2

fuente: https://community.cloudera.com/t5/Support-Questions/How-to-import-a-data-from-URL-through-pyspark/td-p/187772

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Para el tratamiento de datos, es imprescindible que estos tengan una buena calidad. En este sentido, solemos tratar a priori los dataset, en buscar de datos irrelevantes, duplicados, errores de sintaxis, valores ausentes y outliers. Cada uno de estos aspectos tiene un tratamiento diferente, y que en muchos casos, depende del científico de datos que los maneje. Por ejemplo, en el caso de valores ausentes se podría plantear la conveniencia de eliminar los registros con valores ausentes (drop), la imputación de valores a través de funciones creadas al efecto o la consideración de los valores ausentes como una categoría más dentro de un factor (por ejemplo, los usuarios que rellenan formularios pueden no declarar cierta información especialmente sensible por desconfianza o con una intención maliciosa, por lo que esos valores ausentes adquieres especial relevancia y resultan informativos)

fuente: https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

Nos remitimos al apartado anterior "Preparación del dataset", donde realizamos la importación de datos y la limpieza de los mismos.

## Quinta tarea: Media de la diferencia de puntos por año

In [31]:
puntos = df6.select('año','puntosLocal','puntosVisitante')
puntostotales = puntos.withColumn('puntos totales',puntos['puntosLocal']+puntos['puntosVisitante'])

puntostotales.groupBy("año").agg({"puntos totales":"avg"}).show()


+----+-------------------+
| año|avg(puntos totales)|
+----+-------------------+
|2016| 207.56714178544635|
|2012| 192.98778833107193|
|2017| 212.68244084682442|
|2014| 202.14917541229386|
|2013|  197.5868580060423|
|2009|  200.2758358662614|
|2011| 198.06892655367233|
|2008| 199.08408408408408|
|2007|  197.2017543859649|
|2015|  200.5049279757392|
|2010|  200.1423164269493|
+----+-------------------+



## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

Como se puede observar a continuación, el número de partidos jugados por los distintos equipos a lo largo de los 10 años estudiados es muy dispar.

In [32]:
#Total de partidos
matches = df6.select('local','visitante')
matches.count()

12897

In [33]:
partidosHome = matches.groupby('local').count().show()


+--------------------+-----+
|               local|count|
+--------------------+-----+
|        Phoenix Suns|  412|
|      Boston Celtics|  467|
|    Dallas Mavericks|  431|
|New Orleans Pelicans|  166|
|       Brooklyn Nets|  217|
|     New York Knicks|  412|
| New Orleans Hornets|  250|
|   Memphis Grizzlies|  433|
|Minnesota Timberw...|  402|
|  Los Angeles Lakers|  450|
|Golden State Warr...|  445|
|       Orlando Magic|  431|
|   Charlotte Bobcats|  283|
|Los Angeles Clippers|  431|
|     Detroit Pistons|  415|
|       Chicago Bulls|  436|
|     Milwaukee Bucks|  413|
| Cleveland Cavaliers|  449|
|      Indiana Pacers|  434|
|  Washington Wizards|  421|
+--------------------+-----+
only showing top 20 rows



In [191]:
partidosVisit =matches.groupby('visitante').count().show()

+--------------------+-----+
|           visitante|count|
+--------------------+-----+
|        Phoenix Suns|  413|
|      Boston Celtics|  463|
|    Dallas Mavericks|  436|
|New Orleans Pelicans|  166|
|       Brooklyn Nets|  218|
|     New York Knicks|  413|
| New Orleans Hornets|  249|
|Minnesota Timberw...|  402|
|   Memphis Grizzlies|  434|
|  Los Angeles Lakers|  447|
|Golden State Warr...|  440|
|       Orlando Magic|  432|
|   Charlotte Bobcats|  283|
|Los Angeles Clippers|  430|
|     Detroit Pistons|  414|
|       Chicago Bulls|  437|
|     Milwaukee Bucks|  414|
| Cleveland Cavaliers|  452|
|      Indiana Pacers|  434|
|  Washington Wizards|  423|
+--------------------+-----+
only showing top 20 rows



## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

In [34]:
cleveland = df6.select('ganador', 'mes').filter(df6.ganador == 'Cleveland Cavaliers').filter(df6.mes == 'Jan')
cleveland.count()

83

A lo largo de todo el periodo considerdo (10 temporadas) los Cleveland Cavaliers han ganado 83 partidos en los meses de enero

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [35]:
partidosganados = df6.select('local', 'ganador').filter(df6.ganador == 'Golden State Warriors')


In [36]:
partidosganados.count()

523

Los Warriors han **ganado** 523 partidos en total, de los cuales:

In [37]:
comoLocal = partidosganados.filter(partidosganados.local =='Golden State Warriors')

In [38]:
comoLocal.count()

308

Los Warriors han ganadado 308 partidos como local frente a 215 partidos como visitante. Por tanto, los Warriors son mejores como equipo local

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

In [85]:
partidos = df6.select('ganador','año')

In [104]:
#Suma de partidos ganados por cada equipo.
victorias = partidos.groupby('año','ganador').count()
victorias = victorias.select(col('año').alias('año'),col('ganador').alias('ganador'),col('count').alias('vict'))
victorias.show()


+----+--------------------+----+
| año|             ganador|vict|
+----+--------------------+----+
|2009|       Atlanta Hawks|  51|
|2011|      Boston Celtics|  38|
|2016|   Charlotte Hornets|  53|
|2017|    Sacramento Kings|  18|
|2010|       Orlando Magic|  67|
|2014|       Orlando Magic|  26|
|2012|      Indiana Pacers|  63|
|2009|       Orlando Magic|  70|
|2012|   Charlotte Bobcats|  14|
|2015|     Milwaukee Bucks|  39|
|2012|     Detroit Pistons|  35|
|2014|      Boston Celtics|  23|
|2017| Cleveland Cavaliers|  39|
|2010|  Washington Wizards|  24|
|2015|      Indiana Pacers|  44|
|2009|    Sacramento Kings|  23|
|2016|Oklahoma City Thu...|  64|
|2010|     New Jersey Nets|  18|
|2013|  Philadelphia 76ers|  29|
|2015|  Philadelphia 76ers|  17|
+----+--------------------+----+
only showing top 20 rows



In [105]:
#Ganador por temporada
victoriasPorAño = victorias.groupby('año').agg({"vict":"max"}).show()

+----+---------+
| año|max(vict)|
+----+---------+
|2016|       87|
|2012|       82|
|2017|       54|
|2014|       73|
|2013|       85|
|2009|       81|
|2011|       53|
|2008|       84|
|2007|       26|
|2015|       88|
|2010|       71|
+----+---------+



In [ ]:
#Join para asociar las líneas de victoriasPorAño con el nombre del equipo

In [107]:
victorias.join(victoriasPorAño,
               on=victoriasPorAño['max(vict)'],how='leftsemi').show()

TypeError: 'NoneType' object is not subscriptable

Los Warriors son los que más temporadas han ganado en los 10 años estudiados (2015,2016 y 2017)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

In [110]:
'Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com'